In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

spark= SparkSession \
       .builder \
       .appName("daily_practice") \
       .config('spark.ui.port', '4050') \
       .getOrCreate()


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,543 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,155 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all

In [2]:
from google.colab import output
output.serve_kernel_port_as_window(4050, path ='/jobs/index.html')

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

# 🚩 **Day 6 - 2025/04/29**

### Q1: Multi-level Aggregation and Prediction
Build a recommendation engine system that analyzes purchase patterns. Your input is a series of user-product interactions with timestamps and you need to find products that are frequently bought together within a 30-day window, then calculate the probability of future purchases based on past behavior.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, count, sum, when, collect_list, explode, array
from pyspark.sql.window import Window

# User purchase history data
purchases = spark.createDataFrame([
    (101, "A123", "2024-01-15", 29.99),
    (101, "B456", "2024-01-18", 15.50),
    (101, "C789", "2024-02-02", 45.00),
    (102, "A123", "2024-01-05", 29.99),
    (102, "D012", "2024-01-20", 18.75),
    (102, "B456", "2024-02-10", 15.50),
    (103, "B456", "2024-02-01", 15.50),
    (103, "C789", "2024-02-05", 45.00),
    (103, "E345", "2024-02-15", 60.25),
    (104, "A123", "2024-01-10", 29.99),
    (104, "C789", "2024-01-25", 45.00),
    (104, "E345", "2024-02-20", 60.25)
], ["user_id", "product_id", "purchase_date", "amount"])
```

Expected output (showing product pairs that are purchased together and their future purchase probability):
```
+----------+----------+--------------------+--------------------+
|product_1 |product_2 |purchase_frequency  |purchase_probability|
+----------+----------+--------------------+--------------------+
|A123      |C789      |2                   |0.667               |
|B456      |C789      |2                   |0.667               |
|A123      |B456      |1                   |0.333               |
...
+----------+----------+--------------------+--------------------+
```

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, count, sum, when, collect_list, explode, array
from pyspark.sql.window import Window

# User purchase history data
purchases = spark.createDataFrame([
    (101, "A123", "2024-01-15", 29.99),
    (101, "B456", "2024-01-18", 15.50),
    (101, "C789", "2024-02-02", 45.00),
    (102, "A123", "2024-01-05", 29.99),
    (102, "D012", "2024-01-20", 18.75),
    (102, "B456", "2024-02-10", 15.50),
    (103, "B456", "2024-02-01", 15.50),
    (103, "C789", "2024-02-05", 45.00),
    (103, "E345", "2024-02-15", 60.25),
    (104, "A123", "2024-01-10", 29.99),
    (104, "C789", "2024-01-25", 45.00),
    (104, "E345", "2024-02-20", 60.25)
], ["user_id", "product_id", "purchase_date", "amount"])

purchases.printSchema()

purchases.show()

root
 |-- user_id: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- purchase_date: string (nullable = true)
 |-- amount: double (nullable = true)

+-------+----------+-------------+------+
|user_id|product_id|purchase_date|amount|
+-------+----------+-------------+------+
|    101|      A123|   2024-01-15| 29.99|
|    101|      B456|   2024-01-18|  15.5|
|    101|      C789|   2024-02-02|  45.0|
|    102|      A123|   2024-01-05| 29.99|
|    102|      D012|   2024-01-20| 18.75|
|    102|      B456|   2024-02-10|  15.5|
|    103|      B456|   2024-02-01|  15.5|
|    103|      C789|   2024-02-05|  45.0|
|    103|      E345|   2024-02-15| 60.25|
|    104|      A123|   2024-01-10| 29.99|
|    104|      C789|   2024-01-25|  45.0|
|    104|      E345|   2024-02-20| 60.25|
+-------+----------+-------------+------+



In [18]:
from pyspark.sql.functions import col, abs, datediff, least, greatest


# convert to date
purchases = purchases.withColumn("purchase_date", F.to_date("purchase_date"))

p1 = purchases.alias("p1")
p2 = purchases.alias("p2")

# self join the purchase data for the same user where the difference between purchases is less 30 days and not identical products
product_pairs = p1.join(
    p2,
    on="user_id"
).filter(
    (col("p1.product_id") != col("p2.product_id")) &
    (abs(datediff(col("p1.purchase_date"), col("p2.purchase_date"))) <= 30)
)

# we cannot directly use the p1.product_id and p2.product_id to generate product pairs as there can be duplicate pairs
# A123-B456 is same as B456-A123
# therefore we use least and greatest function which returns the min and max values from a list
product_pairs = product_pairs.withColumn("product_1", least(col("p1.product_id"), col("p2.product_id"))) \
                             .withColumn("product_2", greatest(col("p1.product_id"), col("p2.product_id")))

# for each user shows us only the distinct pair of products
unique_product_pairs = product_pairs.select("user_id", "product_1", "product_2").distinct()

purchase_frequency = unique_product_pairs.groupby(["product_1", "product_2"]).agg(count("user_id").alias("purchase_frequency"))
purchase_frequency.show()

# purchase_probability = frequency / product_1_user_count
product_1_user_count = unique_product_pairs.groupBy("product_1").agg(count("user_id").alias("product_1_user_count"))

+---------+---------+------------------+
|product_1|product_2|purchase_frequency|
+---------+---------+------------------+
|     A123|     C789|                 2|
|     B456|     C789|                 2|
|     A123|     D012|                 1|
|     A123|     B456|                 1|
|     B456|     E345|                 1|
|     C789|     E345|                 2|
|     B456|     D012|                 1|
+---------+---------+------------------+



We're aiming to answer:

> **“Given that a user bought `product_1`, what is the probability that they also bought `product_2` within 30 days?”**

This is a **conditional probability**, written as:

\[
P(\text{product}_2 \mid \text{product}_1) = \frac{\text{Users who bought both}}{\text{Users who bought product}_1}
\]

So:

- The **numerator** is the count of users who bought both products within the 30-day window.
- The **denominator** is the number of users who bought `product_1` at all — which is what you computed in the `product_1_user_count`.

In [21]:
final_result = purchase_frequency.join(product_1_user_count, on="product_1") \
    .withColumn("purchase_probability", F.round(F.col("purchase_frequency") / F.col("product_1_user_count"), 3)) \
    .select("product_1", "product_2", "purchase_frequency", "purchase_probability")

final_result.show()

+---------+---------+------------------+--------------------+
|product_1|product_2|purchase_frequency|purchase_probability|
+---------+---------+------------------+--------------------+
|     B456|     D012|                 1|                0.25|
|     B456|     E345|                 1|                0.25|
|     B456|     C789|                 2|                 0.5|
|     A123|     B456|                 1|                0.25|
|     A123|     D012|                 1|                0.25|
|     A123|     C789|                 2|                 0.5|
|     C789|     E345|                 2|                 1.0|
+---------+---------+------------------+--------------------+



### Q2: Streaming Data Quality Monitoring
Implement a streaming data quality monitoring system that validates incoming data against predefined rules, detects outliers, and tracks data drift over time. The system should maintain a dynamic baseline and alert when data deviates significantly from historical patterns.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, abs, count, when, window
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, DoubleType

# Schema for sensor readings
schema = StructType([
    StructField("device_id", StringType(), False),
    StructField("timestamp", TimestampType(), False),
    StructField("temperature", DoubleType(), True),
    StructField("humidity", DoubleType(), True),
    StructField("pressure", DoubleType(), True)
])

# Rules for data quality checks
rules = [
    {"field": "temperature", "min": -30.0, "max": 50.0},
    {"field": "humidity", "min": 0.0, "max": 100.0},
    {"field": "pressure", "min": 900.0, "max": 1100.0}
]

# Sample of historical baseline statistics (pre-calculated)
baseline_stats = spark.createDataFrame([
    ("temperature", 22.5, 3.2),
    ("humidity", 45.2, 8.5),
    ("pressure", 1013.2, 5.1)
], ["field", "baseline_mean", "baseline_stddev"])

# Simulate streaming data
sensor_data = spark.readStream.format("rate").option("rowsPerSecond", 10).load()
sensor_data = sensor_data.selectExpr(
    "cast(value % 5 + 1 as string) as device_id",
    "timestamp",
    "cast(value % 100 as double) as temperature",
    "cast(value % 100 * 0.8 + 10 as double) as humidity",
    "cast(value % 10 + 1000 as double) as pressure"
)
```

The solution should:
1. Apply rule-based validation for each field
2. Detect statistical outliers (values > 3 standard deviations from baseline)
3. Track data drift using a sliding window and compare with the baseline
4. Generate alerts when significant deviations are detected

### Q3: Graph Analysis for Fraud Detection
Implement a fraud detection system using graph analysis techniques to identify suspicious patterns in transaction data. The system should detect cycles (money moving in circles), unusual transaction frequencies, and anomalous transaction amounts.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, collect_list, struct, to_json, from_json
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, TimestampType, DoubleType

# Transaction data
transactions = spark.createDataFrame([
    ("T1001", "ACC_A", "ACC_B", "2024-03-01 08:32:15", 5000.00),
    ("T1002", "ACC_B", "ACC_C", "2024-03-01 09:15:42", 4800.00),
    ("T1003", "ACC_C", "ACC_D", "2024-03-01 10:05:30", 4700.00),
    ("T1004", "ACC_D", "ACC_A", "2024-03-01 11:22:18", 4650.00),  # Cycle completes
    ("T1005", "ACC_E", "ACC_F", "2024-03-01 13:45:22", 12500.00),
    ("T1006", "ACC_G", "ACC_H", "2024-03-01 14:12:55", 8750.00),
    ("T1007", "ACC_H", "ACC_I", "2024-03-01 15:30:17", 8500.00),
    ("T1008", "ACC_I", "ACC_G", "2024-03-01 16:25:40", 8250.00),  # Cycle completes
    ("T1009", "ACC_J", "ACC_K", "2024-03-01 17:10:33", 3000.00),
    ("T1010", "ACC_L", "ACC_J", "2024-03-01 18:05:12", 9500.00)
], ["transaction_id", "from_account", "to_account", "timestamp", "amount"])

# Account risk scores (precomputed based on history)
account_risk = spark.createDataFrame([
    ("ACC_A", 0.2),
    ("ACC_B", 0.3),
    ("ACC_C", 0.4),
    ("ACC_D", 0.6),
    ("ACC_E", 0.1),
    ("ACC_F", 0.2),
    ("ACC_G", 0.7),
    ("ACC_H", 0.8),
    ("ACC_I", 0.7),
    ("ACC_J", 0.3),
    ("ACC_K", 0.2),
    ("ACC_L", 0.1)
], ["account_id", "risk_score"])
```

Your solution should:
1. Detect transaction cycles (money flowing in circular patterns)
2. Identify accounts with high transaction frequency within short time periods
3. Flag transactions with amounts that gradually decrease in a cycle
4. Calculate a fraud risk score based on graph pattern detection
5. Output suspicious transaction patterns ranked by risk

### Q4: Real-time Anomaly Detection with Dynamic Thresholds
Build a system that detects anomalies in time series data from multiple sensors, with dynamically adjusting thresholds based on seasonality patterns and contextual factors. The system should learn normal behavior patterns and adjust sensitivity based on time of day and operational conditions.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, window, avg, stddev, expr, hour, date_format
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType

# Schema for sensor readings
schema = StructType([
    StructField("sensor_id", StringType(), False),
    StructField("timestamp", TimestampType(), False),
    StructField("reading", DoubleType(), False),
    StructField("location", StringType(), True),
    StructField("operational_mode", IntegerType(), True)
])

# Historical summary statistics by hour and operational mode
historical_patterns = spark.createDataFrame([
    ("S1", 0, 1, 22.5, 3.2),  # Sensor S1, midnight hour, mode 1
    ("S1", 0, 2, 21.2, 2.5),  # Sensor S1, midnight hour, mode 2
    ("S1", 12, 1, 35.8, 4.5), # Sensor S1, noon hour, mode 1
    ("S1", 12, 2, 32.5, 3.8), # Sensor S1, noon hour, mode 2
    ("S2", 0, 1, 45.2, 5.1),
    ("S2", 0, 2, 42.5, 4.8),
    ("S2", 12, 1, 62.3, 7.2),
    ("S2", 12, 2, 58.7, 6.5)
], ["sensor_id", "hour_of_day", "operational_mode", "historical_mean", "historical_stddev"])

# Context factors that influence thresholds
context_adjustments = spark.createDataFrame([
    ("S1", "ZONE_A", 1.2),  # Multiply threshold by 1.2 for S1 in ZONE_A
    ("S1", "ZONE_B", 0.9),  # Reduce threshold by 10% for S1 in ZONE_B
    ("S2", "ZONE_A", 1.1),
    ("S2", "ZONE_B", 0.8)
], ["sensor_id", "location", "threshold_multiplier"])

# Sample batch of incoming sensor data for testing
current_readings = spark.createDataFrame([
    ("S1", "2024-04-01 00:05:23", 25.8, "ZONE_A", 1),
    ("S1", "2024-04-01 00:15:45", 24.2, "ZONE_A", 1),
    ("S1", "2024-04-01 00:25:12", 35.7, "ZONE_A", 1),  # Anomaly!
    ("S1", "2024-04-01 12:10:33", 36.2, "ZONE_B", 1),
    ("S1", "2024-04-01 12:20:56", 50.5, "ZONE_B", 1),  # Anomaly!
    ("S2", "2024-04-01 00:03:41", 46.5, "ZONE_A", 2),
    ("S2", "2024-04-01 00:13:25", 80.2, "ZONE_A", 2),  # Anomaly!
    ("S2", "2024-04-01 12:07:19", 61.8, "ZONE_B", 2)
], ["sensor_id", "timestamp", "reading", "location", "operational_mode"])
```

Your solution should:
1. Detect point anomalies using dynamic Z-score thresholds adjusted by time and context
2. Identify contextual anomalies by comparing with historical patterns
3. Calculate an anomaly severity score for each detected anomaly
4. Consider operational mode and location context when determining thresholds

### Q5: Incremental ML Model Monitoring and Retraining
Implement a system that monitors ML model performance in production, detects model drift, and triggers retraining when needed. The system should analyze prediction errors, feature distributions, and business KPIs to determine when model retraining is necessary.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, stddev, sum, abs, expr, datediff, to_date
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType, BooleanType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml.classification import LogisticRegression

# Model metadata - current production model
model_metadata = {
    "model_id": "CLF-2024-03",
    "deployment_date": "2024-03-01",
    "features": ["feature_1", "feature_2", "feature_3", "feature_4"],
    "baseline_auc": 0.85,
    "baseline_precision": 0.82,
    "baseline_recall": 0.78,
    "baseline_feature_distributions": {
        "feature_1": {"mean": 0.52, "stddev": 0.21},
        "feature_2": {"mean": 25.4, "stddev": 12.3},
        "feature_3": {"mean": 0.65, "stddev": 0.18},
        "feature_4": {"mean": 42.1, "stddev": 15.6}
    },
    "performance_threshold": 0.05  # Acceptable performance degradation threshold
}

# Historical model predictions and outcomes
historical_predictions = spark.createDataFrame([
    ("2024-03-05", "user_123", 0.82, 1, 0.51, 24.8, 0.67, 43.2, 500),
    ("2024-03-05", "user_456", 0.35, 0, 0.48, 22.5, 0.62, 40.1, 250),
    # ... more historical data
], ["prediction_date", "user_id", "probability", "actual_outcome",
    "feature_1", "feature_2", "feature_3", "feature_4", "transaction_amount"])

# Recent model predictions and outcomes (for drift detection)
recent_predictions = spark.createDataFrame([
    ("2024-04-15", "user_789", 0.78, 1, 0.55, 26.2, 0.68, 44.5, 650),
    ("2024-04-15", "user_101", 0.42, 1, 0.49, 21.8, 0.61, 38.9, 300),
    ("2024-04-16", "user_112", 0.83, 0, 0.56, 27.5, 0.71, 45.2, 700),
    ("2024-04-16", "user_131", 0.39, 1, 0.50, 23.2, 0.63, 41.3, 400),
    ("2024-04-17", "user_415", 0.81, 1, 0.54, 26.8, 0.69, 44.8, 600),
    ("2024-04-17", "user_167", 0.32, 0, 0.47, 21.5, 0.60, 39.5, 200),
    ("2024-04-18", "user_189", 0.85, 1, 0.57, 28.2, 0.72, 46.0, 750),
    ("2024-04-18", "user_215", 0.37, 0, 0.48, 22.0, 0.62, 40.5, 350),
    ("2024-04-19", "user_227", 0.76, 0, 0.53, 25.5, 0.67, 43.5, 550),
    ("2024-04-19", "user_239", 0.34, 1, 0.46, 21.2, 0.59, 38.7, 150),
    # Several predictions with different feature values to test feature drift
    ("2024-04-20", "user_241", 0.77, 1, 0.65, 35.2, 0.80, 58.5, 800),
    ("2024-04-20", "user_253", 0.81, 0, 0.67, 36.8, 0.82, 59.2, 850),
    ("2024-04-20", "user_265", 0.79, 1, 0.66, 35.5, 0.81, 58.8, 820),
    ("2024-04-20", "user_277", 0.82, 1, 0.69, 37.0, 0.83, 60.0, 900),
    ("2024-04-20", "user_289", 0.80, 0, 0.68, 36.5, 0.82, 59.5, 880)
], ["prediction_date", "user_id", "probability", "actual_outcome",
    "feature_1", "feature_2", "feature_3", "feature_4", "transaction_amount"])
```

Hint: Your solution should implement various drift detection techniques, including:
1. Performance drift (AUC, precision, recall degradation)
2. Feature distribution drift (statistical tests)
3. Prediction distribution drift
4. Error analysis by features and segments
5. A retraining decision algorithm that considers multiple factors